In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Changing Paragraph to sentences

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# Download the necessary NLTK data files
nltk.download('punkt')

# Load the dataset
file_path = '/kaggle/input/task4-dataset/task4/paragraphs.xlsx'
df = pd.read_excel(file_path)

# Ensure the columns are named appropriately
paragraph_headings = df.iloc[:, 0]  # Assuming the headings are in the first column
paragraphs = df.iloc[:, 2]  # Adjust the index if the paragraphs are in another column

# Initialize a list to store the sentences with their corresponding paragraph heading
sentences_with_headings = []

# Process each paragraph
for heading, paragraph in zip(paragraph_headings, paragraphs):
    if isinstance(paragraph, str):  # Check if the paragraph is a string
        # Tokenize the paragraph into sentences
        sentences = sent_tokenize(paragraph)
        # Append the sentences and their corresponding heading to the list
        for sentence in sentences:
            sentences_with_headings.append((heading, sentence))
    else:
        # Handle non-string paragraphs (e.g., None, NaN, etc.)
        sentences_with_headings.append((heading, paragraph))

# Create a new DataFrame with the paragraph heading and sentences
sentences_df = pd.DataFrame(sentences_with_headings, columns=['Paragraph Heading', 'Sentence'])

# Save the sentences to a new Excel file
output_file_path = 'sentencefrompara.xlsx'
sentences_df.to_excel(output_file_path, index=False)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00a 0:00:01


# Using a pretrained model for converting sentence to embeddings and then performing cosine similarity

In [4]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
embedder = SentenceTransformer("all-MiniLM-L6-v2")
def semantic_search(query,corpus_embeddings ,top_k=5):
#     corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    top_k = min(top_k, len(corpus))

    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    top_paragraphs = [corpus[idx] for idx in top_results[1]]
    return top_paragraphs

def answer_question(question, top_paragraphs):
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
    
    best_answer = None
    best_score = 0
    best_paragraph = ""

    for paragraph in top_paragraphs:
        answer = qa_pipeline({'question': question, 'context': paragraph})
        if answer['score'] > best_score:
            best_score = answer['score']
            best_answer = answer['answer']
            best_paragraph = paragraph

    return best_paragraph, best_answer, best_score

corpus = sentences_df['Sentence'].tolist()
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
np.save('corpus_embeddings.npy', corpus_embeddings.cpu().numpy())  # Save embeddings as a numpy array
def load_embeddings(embedding_file='corpus_embeddings.npy'):
    return torch.tensor(np.load(embedding_file))

##for loading embedding use this function and say corpus_embeddings=load_embeddings(embedding_file='corpus_embedding.npy')

2024-05-23 15:19:42.451881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 15:19:42.451982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 15:19:42.587628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11996 [00:00<?, ?it/s]

# Functions to get answer

In [7]:
def answer_question(question, bestparagraph):
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
    
    answers = []
#     for paragraph in top_paragraphs:
    answer = qa_pipeline({'question': question, 'context': bestparagraph})
    answers.append((paragraph, answer['answer'], answer['score']))

    # Sort answers by score in descending order
#     answers = sorted(answers, key=lambda x: x[2], reverse=True)
    return answers
#  the below comment if you want to input a query
queries = [
    "What event was the first HDTV broadcast in Europe?",
    "Who translated German chorales into English?",
    "What entity controls the schools in Hyderabad?"
]
for query in queries:
        if query.lower() == 'exit':
            break
        top_paragraphs = semantic_search(query, corpus_embeddings)
        
        print("\nTop 5 paragraphs:")
        for i, para in enumerate(top_paragraphs, 1):
            print(f"{i}: {para}")

        question = query
        answers = answer_question(question, top_paragraphs[0])
        
        print("\nAnswers based on top paragraphs:")
        for paragraph, answer, score in answers:
            print(f"Answer: {answer}\nScore: {score:.4f}\n")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Top 5 paragraphs:
1: The first HDTV transmissions in Europe, albeit not direct-to-home, began in 1990, when the Italian broadcaster RAI used the HD-MAC and MUSE HDTV technologies to broadcast the 1990 FIFA World Cup.
2: The first HDTV transmissions in Europe, albeit not direct-to-home, began in 1990, when the Italian broadcaster RAI used the HD-MAC and MUSE HDTV technologies to broadcast the 1990 FIFA World Cup.
3: The first HDTV transmissions in Europe, albeit not direct-to-home, began in 1990, when the Italian broadcaster RAI used the HD-MAC and MUSE HDTV technologies to broadcast the 1990 FIFA World Cup.
4: The first HDTV transmissions in Europe, albeit not direct-to-home, began in 1990, when the Italian broadcaster RAI used the HD-MAC and MUSE HDTV technologies to broadcast the 1990 FIFA World Cup.
5: The first HDTV transmissions in Europe, albeit not direct-to-home, began in 1990, when the Italian broadcaster RAI used the HD-MAC and MUSE HDTV technologies to broadcast the 1990 FI

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Answers based on top paragraphs:
Answer: 1990 FIFA World Cup
Score: 0.5656



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


Answers based on top paragraphs:
Answer: Catherine Winkworth
Score: 0.9717



Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Top 5 paragraphs:
1: Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan.
2: Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan.
3: Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan.
4: Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan.
5: Institutes in Hyderabad include the National Institute of Rural Development, the Indian School of Business, the Institute of Public Enterprise, the Administrative Staff College of India and the Sardar Vallabhbhai Patel National Police Academy.


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Answers based on top paragraphs:
Answer: Central Board of Secondary Education
Score: 0.6207



# For interactive search

In [9]:
if __name__ == "__main__":
    while True:
        query = input("Enter a query sentence (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        top_paragraphs = semantic_search(query, corpus_embeddings)
        
        print("\nTop 5 paragraphs:")
        for i, para in enumerate(top_paragraphs, 1):
            print(f"{i}: {para}")

        question = query
        answers = answer_question(question, top_paragraphs[0])
        
        print("\nAnswers based on top paragraphs:")
        for paragraph, answer, score in answers:
            print(f"Answer: {answer}\nScore: {score:.4f}\n")


Enter a query sentence (or type 'exit' to quit):  Who sent her a letter asking that she reconsider the Pepsi deal?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Top 5 paragraphs:
1: Pepsi revoked the commercial and canceled her sponsorship contract.
2: Pepsi revoked the commercial and canceled her sponsorship contract.
3: Pepsi revoked the commercial and canceled her sponsorship contract.
4: Pepsi revoked the commercial and canceled her sponsorship contract.
5: In 2012, Beyoncé signed a $50 million deal to endorse Pepsi.


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Answers based on top paragraphs:
Answer: Pepsi
Score: 0.6519



...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Top 5 paragraphs:
1: Blair announced in September 2006 that he would quit as leader within the year, though he had been under pressure to quit earlier than May 2007 in order to get a new leader in place before the May elections which were expected to be disastrous for Labour.
2: Blair announced in September 2006 that he would quit as leader within the year, though he had been under pressure to quit earlier than May 2007 in order to get a new leader in place before the May elections which were expected to be disastrous for Labour.
3: Blair announced in September 2006 that he would quit as leader within the year, though he had been under pressure to quit earlier than May 2007 in order to get a new leader in place before the May elections which were expected to be disastrous for Labour.
4: Blair announced in September 2006 that he would quit as leader within the year, though he had been under pressure to quit earlier than May 2007 in order to get a new leader in place before the May elec

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Answers based on top paragraphs:
Answer: Blair announced in September 2006 that he would quit as leader within the year
Score: 0.1347



Enter a query sentence (or type 'exit' to quit):  exit


# Tried gradio throwing error on kaggle

In [10]:
!pip install gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=bb4305c4c8eeba1cdfd80db373d25e867034196aad1c615d2ffa599a973c5e53
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting unins

Did implementation on gradio

In [11]:
import gradio as gr
def process_query(query):
    if query.lower() == 'exit':
        return [], []

    # Perform semantic search
    top_paragraphs = semantic_search(query, corpus_embeddings)
    
    # Answer the question based on the top paragraph
    question = query
    answers = answer_question(question, top_paragraphs[0])
    
    return top_paragraphs, answers

def gradio_interface(query):
    top_paragraphs, answers = process_query(query)
    
    # Format the output
    paragraphs_output = "\n".join([f"{i+1}: {para}" for i, para in enumerate(top_paragraphs)])
    answers_output = "\n\n".join([f"Answer: {answer}\nScore: {score:.4f}" for _, answer, score in answers])
    
    return paragraphs_output, answers_output

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "text"],
    title="Semantic Search and Question Answering",
    description="Enter a query to get the top 5 paragraphs and answers based on the top paragraph."
)

# Launch the Gradio interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a73a15813e923ff5e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
